# Create a Local Ethernet (Layer 2) Network

This notebook shows how to create an isolated local Ethernet and connect compute nodes to it.  


## Import the FABlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                         
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,f8a6e0b0-ad14-47cb-9764-74c20ef3e4fc
Bastion Username,durbek_gafurov_0000000854
Bastion Private Key File,/home/fabric/work/fabric_config/bastionD
Bastion Host,bastion-1.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/.ssh/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/.ssh/slice_key
Slice Private Key Passphrase,None


## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Address,Location,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,RAM Available,RAM Capacity,RAM Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated
STAR,"710 North Lake Shore Dr,Chicago, IL 60611","(41.89537135, -87.61663220067463)",6,12,384,384,0,3072,3072,0,121200,121200,0,762,762,0,2,2,0,6,6,0,20,20,0,6,6,0,6,6,0
MASS,"100 Bigelow Street,Holyoke MA 01040","(42.202493000000004, -72.60787662257826)",3,6,178,192,14,1488,1536,48,60534,60600,66,375,381,6,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
SALT,"572 Delong St,Salt Lake City, UT 84104","(40.75707505789612, -111.95346637770317)",3,6,174,192,18,1496,1536,40,60510,60600,90,375,381,6,1,2,1,2,2,0,10,10,0,2,2,0,3,3,0
NCSA,"1725 S Oak St.,Champaign, IL 61820","(40.1035624, -88.2415105)",3,6,168,192,24,1504,1536,32,60500,60600,100,378,381,3,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
UTAH,"875 South West Temple,Salt Lake City, UT 84101","(40.7618296, -111.8939542)",5,10,254,320,66,2424,2560,136,116110,116400,290,625,635,10,2,2,0,4,4,0,16,16,0,4,4,0,5,5,0
WASH,"The Bexley, 1761 Old Meadow Road, McLean, VA 22102, United States of America","(38.91930235, -77.21183383681088)",3,6,90,192,102,1152,1536,384,59450,60600,1150,361,381,20,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
DALL,"1950 N Stemmons Fwy,Dallas, TX 75207","(32.8002714, -96.8198113)",3,6,166,192,26,1432,1536,104,60450,60600,150,374,381,7,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
MAX,,"(0, 0)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
MICH,"2530 Draper Dr,Ann Arbor, MI 48109","(42.2931086, -83.7101319)",3,6,146,192,46,1416,1536,120,59430,60600,1170,363,381,18,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
TACC,"10100 Burnet Rd,Austin, TX 78758","(30.3899405, -97.7261806879021)",5,10,260,320,60,2200,2560,360,115444,116400,956,619,635,16,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0


## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add an `l2network` to the slice and pass the list of interfaces you want connected to this Ethernet. If all interfaces in the list are located on the same site, the network will automatically be a local Ethernet.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on the same site you can specify the name of the site in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a random site name that can be used for both nodes.

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [3]:
slice_name = 'Experiment 1.2'

site1 = 'GPN'  
site2 = 'SALT'

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
network1_2_name='net1_2'
network2_3_name='net2_3'
node1_nic_name = 'nic1'
node2_1_nic_name = 'nic2_1'
node2_3_nic_name = 'nic2_3'
node3_nic_name = 'nic3'
image='default_ubuntu_20'

cores = 2
ram = 8
disk = 10

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site1, cores=cores, ram=ram, disk=disk)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    node1.set_image(image)
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site1, cores=cores, ram=ram, disk=disk)
    iface2_1 = node2.add_component(model='NIC_Basic', name=node2_1_nic_name).get_interfaces()[0]
    iface2_3 = node2.add_component(model='NIC_Basic', name=node2_3_nic_name).get_interfaces()[0]
    node2.set_image(image)

    # Node3
    node3 = slice.add_node(name=node3_name, site=site2, cores=cores, ram=ram, disk=disk)
    iface3 = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]
    node3.set_image(image)
    
    # Network
    net1_2 = slice.add_l2network(name=network1_2_name, interfaces=[iface1, iface2_1])
    net2_3 = slice.add_l2network(name=network2_3_name, interfaces=[iface2_3, iface3])

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


Retry: 5, Time: 137 sec


ID,77faa968-0460-4733-8882-a3d67dc1d8d0
Name,Experiment 1.2
Lease Expiration (UTC),2023-01-10 17:20:24 +0000
Lease Start (UTC),2023-01-09 17:20:25 +0000
Project ID,f8a6e0b0-ad14-47cb-9764-74c20ef3e4fc
State,Configuring


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
5b18e743-ad1a-416a-aa10-ddc7d12c24c5,Node1,2,8,10,default_ubuntu_20,qcow2,gpn-w4.fabric-testbed.net,GPN,ubuntu,None,Ticketed,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@None,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
90beb2c5-24a0-4265-b4ea-ac9e2bb4384a,Node2,2,8,10,default_ubuntu_20,qcow2,gpn-w4.fabric-testbed.net,GPN,ubuntu,None,Ticketed,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@None,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
76146279-e215-454a-ac66-81ce61efbfe7,Node3,2,8,10,default_ubuntu_20,qcow2,salt-w1.fabric-testbed.net,SALT,ubuntu,None,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@None,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
42cf3d96-6841-4d77-a164-3e531e4eab0e,net1_2,L2,L2Bridge,GPN,None,None,Ticketed,
a69264a2-9678-4716-b54a-5a4a44e72bfe,net2_3,L2,L2STS,None,None,None,Ticketed,


## Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node List

In [ ]:
try:
    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node Details

In [ ]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Interfaces

In [ ]:
try:    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

## Prolong the lease

In [ ]:
end_date = (datetime.datetime.utcnow() + datetime.timedelta(days=36)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice.renew(end_date+"+0000")
    print(datetime.datetime.utcnow(),end_date+"+0000")
except Exception as e:
    print(f"Exception: {e}")

##  Configure IP Addresses

Some experiments use FABRIC layer 2 networks to enable deploying non-IP layer 3 networks.  If this describes your experiment, your nodes and network are ready. You can now login to the nodes and deploy your experiment.

Most users will want to configure IP addresses on their new nodes.  FABlib provides some useful methods to help you configure basic IP addresses. 

### Pick a Subnet

Create a subnet and list of available IP addresses. All objects are Python IP management objects. You can use either IPv4 or IPv6 subnets and addresses.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet1 = IPv4Network("192.168.1.0/24")
    available_ips1 = list(subnet1)[1:]
    subnet2 = IPv4Network("192.168.2.0/24")
    available_ips2 = list(subnet2)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Get the node and the interface you wish to configure.  You can use `node.get_interface` to get the interface that is connected to the specified network.  Then `pop` an IP address from the list of available IPs and call `iface.ip_addr_add` to set the IP and subnet.  

Optionally, use the `node.execute()` method to show the results of adding the IP address.

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_2_name) 
    
    stdout, stderr = node1.execute('sudo apt install net-tools')
    print(stdout, stderr)
    stdout, stderr = node1.execute(f'sudo ifconfig {node1_iface.get_os_interface()} up')
    print(stdout, stderr)

    node1_addr = available_ips1.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet1)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_1_iface = node2.get_interface(network_name=network1_2_name)  
    node2_1_addr = available_ips1.pop(0)
    node2_1_iface.ip_addr_add(addr=node2_1_addr, subnet=subnet1)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_1_iface.get_os_interface()}')
    print (stdout)

    stdout, stderr = node2.execute('sudo apt install net-tools')
    print(stdout, stderr)
    stdout, stderr = node2.execute(f'sudo ifconfig {node2_1_iface.get_os_interface()} up')
    print(stdout, stderr)
    
    node2_3_iface = node2.get_interface(network_name=network2_3_name)  
    node2_3_addr = available_ips2.pop(0)
    node2_3_iface.ip_addr_add(addr=node2_3_addr, subnet=subnet2)
    
    stdout, stderr = node2.execute(f'sudo ifconfig {node2_3_iface.get_os_interface()} up')
    print(stdout, stderr)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_3_iface.get_os_interface()}')
    print (stdout)

except Exception as e:
    print(f"Exception: {e}")

### Configure Node3

Repeat the steps to add the next available IP to the third node.

In [ ]:
try:
    node3 = slice.get_node(name=node3_name)        
    node3_iface = node3.get_interface(network_name=network2_3_name) 
    node3_addr = available_ips2.pop(0)
    node3_iface.ip_addr_add(addr=node3_addr, subnet=subnet2)

    stdout, stderr = node3.execute('sudo apt install net-tools')
    print(stdout, stderr)
    stdout, stderr = node3.execute(f'sudo ifconfig {node2_1_iface.get_os_interface()} up')
    print(stdout, stderr)
    
    stdout, stderr = node3.execute(f'ip addr show {node3_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
node1 = slice.get_node(name=node1_name)
node2 = slice.get_node(name=node2_name) 
node3 = slice.get_node(name=node3_name)
node1_addr = "192.168.1.1"
node2_1_addr = "192.168.1.2"
node2_3_addr = "192.168.2.1"
node3_addr = "192.168.2.2"

## Run the Experiment

We will find the ping round trip time for this pair of sites.


In [ ]:
try:
#     node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 15 {node2_1_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
#     node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node3.execute(f'ping -c 5 {node2_3_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
#     node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node2.execute(f'ping -c 5 {node3_addr}')
    print (stdout)
    print (stderr)
    
except Exception as e:
    print(f"Exception: {e}")

---

## Installing iperf

In [ ]:
config_threads = {}

config_threads[node1] = node1.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
config_threads[node2] = node2.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
config_threads[node3] = node3.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')

for node, thread in config_threads.items():
    stdout, stderr = thread.result()
    print(f"Config thread node {node.get_name()} complete")
    print(stdout)

In [ ]:
# stdout, stderr = node1.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
# print (stdout)

# stdout, stderr = node2.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
# print (stdout)

# stdout, stderr = node3.execute(f'sudo apt update && sudo apt install -y iperf iperf3 ffmpeg')
# print (stdout)

## Setting Parameters

In [ ]:
stdout, stderr = node1.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

In [ ]:
stdout, stderr = node2.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

In [ ]:
stdout, stderr = node3.execute('echo "net.core.rmem_max = 2147483647\nnet.core.wmem_max = 2147483647\nnet.ipv4.tcp_rmem = 4096 87380 2147483647\nnet.ipv4.tcp_wmem = 4096 65536 2147483647\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')
print (stdout)
print (stderr)

In [ ]:
#server
stdout, stderr = node2.execute('tmux new -d \'iperf -s -f K\'') 
print (stdout)
print (stderr)

In [ ]:
#client
stdout, stderr = node1.execute(f'iperf -c {node2_1_addr} -P 50 -w 999M') 
print (stdout)
print (stderr)

In [ ]:
#client
stdout, stderr = node3.execute(f'iperf -c {node2_3_addr} -P 50 -w 999M') 
print (stdout)
print (stderr)

In [ ]:
#client
stdout, stderr = node3.execute(f'iperf -c {node2_3_addr} -P 50 -w 999M') 
print (stdout)
print (stderr)

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")